In [1]:
%defaultDatasource jdbc:h2:mem:db

In [2]:
DROP TABLE IF EXISTS Drug;
DROP TABLE IF EXISTS Pathology;
DROP TABLE IF EXISTS DrugUse;

CREATE TABLE Drug (
  code INT NOT NULL,
  name VARCHAR(200) NOT NULL,
  PRIMARY KEY(code)
) AS SELECT
    code,
    name
FROM CSVREAD('../../../data/faers-2017/drug.csv');

CREATE TABLE Pathology (
  code INT NOT NULL,
  name VARCHAR(200) NOT NULL,
  PRIMARY KEY(code)
) AS SELECT
    code,
    name
FROM CSVREAD('../../../data/faers-2017/pathology.csv');

CREATE TABLE DrugUse (
  idPerson INT NOT NULL,
  codePathology INT NOT NULL,
  codeDrug INT NOT NULL,
  FOREIGN KEY(codePathology)
    REFERENCES Pathology(code)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION,
  FOREIGN KEY(codeDrug)
    REFERENCES Drug(code)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION,
) AS SELECT
    idPerson,
    codePathology,
    codeDrug
FROM CSVREAD('../../../data/faers-2017/drug-use.csv');

CREATE TABLE SideEffect(
  idPerson INT NOT NULL,
  codePathology INT NOT NULL,
  FOREIGN KEY(codePathology)
    REFERENCES Pathology(code)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION
) AS SELECT idPerson, codePathology
FROM CSVREAD('../../../data/faers-2017/sideeffect.csv');

In [3]:
SELECT * FROM Drug;
SELECT * FROM Pathology;
SELECT * FROM DrugUse;
SELECT * FROM SideEffect;

# FDA Adverse Event Reporting System (FAERS)

Considere a base de dados que reúne relatos de efeitos adversos de drogas: [FDA Adverse Event Reporting System (FAERS)](https://open.fda.gov/data/faers/).

As fontes de dados são publicadas e documentadas em: [Latest Quarterly Data Files](https://www.fda.gov/drugs/fda-adverse-event-reporting-system-faers/fda-adverse-event-reporting-system-faers-latest-quarterly-data-files)

# Interação entre medicamentos
Considere como uma interação toda vez que um medicamento A é tomado em conjunto com um medicamento B por uma pessoa. Crie uma consulta que retorne para cada par de medicamento A,B o número de interações, somente para aqueles medicamentos que são tomados em conjunto mais de 30 vezes (passaremos a chamar de **grupo mais 30**).

In [8]:
DROP VIEW IF EXISTS PersonDrugUse;
CREATE VIEW PersonDrugUse AS
SELECT idPerson, codeDrug
    FROM DrugUse
    GROUP BY idPerson,codeDrug;
SELECT * FROM PersonDrugUse;

In [14]:
DROP VIEW IF EXISTS DrugI30Duplicates;
CREATE VIEW DrugI30Duplicates AS
SELECT PDU1.codedrug druga, PDU2.codedrug drugb, COUNT(*) Quantidade
    FROM PersonDrugUse as PDU1, PersonDrugUse as PDU2
    WHERE PDU1.idPerson = PDU2.idPerson and
          PDU1.codeDrug <> PDU2.codeDrug
    GROUP BY PDU1.codedrug, PDU2.codedrug
    HAVING Quantidade > 30;
SELECT * FROM DrugI30Duplicates;

In [ ]:
DROP VIEW IF EXISTS DrugI30;

## Tabela Resultante

Se estiver demorando muito para gerar a tabela, você poderá usar a seguinte tabela resultante pronta. Essa tabela tem todas as interações, você ainda precisará filtrar o **grupo mais 30**.

In [4]:
CREATE TABLE DrugInteraction (
  druga INT NOT NULL,
  drugb INT NOT NULL,
  weight INT NOT NULL,
  PRIMARY KEY(druga, drugb)
) AS SELECT
    druga,
    drugb,
    weight
FROM CSVREAD('../../../data/faers-2017/results/drug-interaction.csv');

SELECT * FROM DrugInteraction;

# Suporte

### Qual o suporte que cada uma das associações tem?

Gere uma consulta que apresente o suporte do **grupo mais 30**. Ordene o resultado por ordem de suporte. 

Observação: Para que o resultado da divisão seja fracionária, converta o numerador em `DOUBLE` usando:
~~~sql
CAST(campo AS DOUBLE)
~~~

In [41]:
SELECT druga, drugb, (CAST(weight AS DOUBLE)/(SELECT COUNT(idPerson) FROM DrugUse)) suporte
    FROM DrugInteraction
    WHERE weight > 30
    ORDER BY suporte DESC;

SELECT SUM(weight) from DrugInteraction;
SELECT COUNT(*) FROM (SELECT COUNT(idPerson) FROM DrugUse GROUP BY idPerson);

# Confiança

### Qual a confiança que cada uma das associações tem?

Gere uma consulta que apresente a confiança do **grupo mais 30**. Ordene o resultado por ordem de confiança.

In [31]:
SELECT DI.druga, DI.drugb, COUNT(DU.codedrug),(CAST(weight AS DOUBLE)/(COUNT(DU.codedrug))) suporte
    FROM DrugInteraction DI, DrugUse DU
    WHERE DI.weight > 30 AND
          DU.codedrug = DI.druga
    GROUP BY DI.druga, DI.drugb
    ORDER BY suporte DESC;

# Efeito Colateral

Para cada medicamento, qual o percentual do medicamento A que tem efeito colateral junto com o medicamento B, quando comparado com seu uso sem o medicamento B.

In [15]:
(SELECT COUNT(*) FROM DrugUse WHERE codedrug = 96)

424

In [27]:
SELECT druga, drugb, weight
    FROM DrugInteraction
    WHERE weight > 30